## Purpose
This notebook is made to help pushing policies produced by TeachMyAgent's experiments on Hugging Face's Hub.
## How to use this notebook
This notebook is broken down into 4 sections:
- **Imports**: import needed packages.
- **Load data**: load results produced by experiments
- **Functions definition**: define functions to compute and push top seeds for each experiment
- **Push policies**: push policies 

Please install TensorflowJS with:
`pip install tensorflowjs`

**Warning: This will update your tensorflow version to >2. Please consider downgrading it back to be between 1.4 and 1.15.5**

# Imports

In [ ]:
import sys
import os
import random
import math
import pylab
import copy
import re
from enum import Enum
from collections import OrderedDict

import numpy as np

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.colorbar as cbar
import seaborn as sns
import imageio

DIV_LINE_WIDTH = 50
print(np.__version__)
print(sys.executable)
sns.set()

In [ ]:
module_path = os.path.abspath(os.path.join('../'))
if module_path not in sys.path:
    sys.path.append(module_path)

from TeachMyAgent.students.run_logs_util import get_run_logs 
from TeachMyAgent.students.package_to_hub import package_to_hub

# Load data

In [ ]:
def get_datasets(rootdir, name_filter=None, rename_labels=False):
    """
        Loads results of experiments.
 
        Results to load can be filtered by their name and each experiment can be associated to a label (usually ACL method's name)
 
        :param rootdir: Directory containing experiments to load (do not forget '/' at the end of the path)
        :param name_filter: String experiments to load must contain
        :param rename_labels: If True, each experiment will be associated to a label (see below). Labels are the names that will appear in plots.
        :type rootdir: str
        :type name_filter: str (or None)
        :type rename_labels: boolean
    """
    _, models_list, _ = next(os.walk(rootdir))
    print(models_list)
    for dir_name in models_list.copy():
        if "ignore" in dir_name:
            models_list.remove(dir_name)
        if name_filter is not None and name_filter not in dir_name:
            models_list.remove(dir_name)         
        
    for i,m_name in enumerate(models_list):           
        print("extracting data for {}...".format(m_name))
        m_id = m_name
        models_saves[m_id] = OrderedDict()
        models_saves[m_id]['data'] = get_run_logs(rootdir+m_name, book_keeping_keys='*', min_len=0)
        print("done")
        if m_name not in labels:
            if not rename_labels:
                labels[m_name] = m_name
            else:
                ##### MODIFY THIS IF YOU ADD A NEW METHOD #####
                if 'ADR' in m_name:
                    labels[m_name] = 'ADR'
                elif 'ALP-GMM' in m_name:
                    labels[m_name] = 'ALP-GMM'
                elif 'Random' in m_name:
                    labels[m_name] = 'Random'
                elif 'Covar-GMM' in m_name:
                    labels[m_name] = 'Covar-GMM'
                elif 'RIAC' in m_name:
                    labels[m_name] = 'RIAC'
                elif 'GoalGAN' in m_name:
                    labels[m_name] = 'GoalGAN'
                elif 'Self-Paced' in m_name:
                    labels[m_name] = 'Self-Paced'
                elif 'Setter-Solver' in m_name:
                    labels[m_name] = 'Setter-Solver'
                elif 'UPPER_BASELINE' in m_name:
                    labels[m_name] = 'UPPER_BASELINE'
                else:
                    labels[m_name] = m_name
                ##### MODIFY THIS IF YOU ADD A NEW METHOD #####
labels = OrderedDict()
models_saves = OrderedDict()

##### MODIFY THIS TO POINT TO YOUR DATA FOLDER #####
data_folder = "/home/cromac/Documents/Projects/TeachMyAgent/ACL-bench_repo/ACL_bench/data/OFFICIAL_RESULTS/BENCHMARK/" # Please use an absolute path
##### MODIFY THIS TO POINT TO YOUR DATA FOLDER #####

get_datasets(data_folder, rename_labels=True, name_filter="parkour_ALP-GMM_walker_type_climbing")

# Functions definition

## Compute top seeds per run

In [ ]:
def get_top_seeds(expe_name, n):
    """
        Calculate best seed of an experiment.
 
        :param expe_name: Experiment's name
        :param metric: Metric to use to calculate best seed
        :type expe_name: str
        :type metric: str
        :return best seed, its metric value, mean of all seeds, std over seeds
    """
    best_seed = -1
    best_seed_value = -1000
    runs_data = models_saves[expe_name]['data']
    metric = 'evaluation return'
    all_values = []
    for run in runs_data:
        if len(run[metric]) > 0:
            data = run[metric][-1]
            all_values.append(data)
        else:
            print("Skipping seed {}: no data".format(run["config"]["seed"]))
            all_values.append(-100)
            
    sorted_indices = np.argsort(-1 * np.array(all_values))[:n]  # Compute top n indices
    top_seeds = []
    means, stds = [], []
    for index in sorted_indices:
        run = runs_data[index]
        top_seeds.append(run["config"]["seed"])
        test_set_size = int(len(run['env_test_rewards'])/len(run['evaluation return']))
        last_evaluation_results = run['env_test_rewards'][-test_set_size:]
        means.append(np.mean(last_evaluation_results))
        stds.append(np.std(last_evaluation_results))
            
    return top_seeds, means, stds

In [ ]:
def push_top_seeds_to_hub(n, experiment_id, base_hyperparameters, token, account):
    label = labels[experiment_id]
    morphology = experiment_id.split("walker_type_")[1]
    hyperparameters = copy.copy(base_hyperparameters)
    hyperparameters["teacher"] = label
    hyperparameters["morphology"] = morphology
    top_seeds, means, stds = get_top_seeds(experiment_id, n)
    for seed, mean_reward, std_reward in zip(top_seeds, means, stds):
        _config = {}
        _config["seed"] = seed
        _config["morphology"] = morphologies_name_mapping[morphology]
        _config["name"] = f"{label}_{hyperparameters['student']}_{_config['morphology']}_s{seed}"
        repo_name = f"{account}/TA_{_config['name']}"
        model_path = os.path.join(data_folder, experiment_id, f"{experiment_id}_s{seed}")
        print(f"Uploading {repo_name}...")
        try:
            package_to_hub(repo_name, _config, model_path, mean_reward, std_reward, hyperparameters, token)
        except Exception as e:
            print(e)

# Push policies

In [ ]:
TOKEN = ''
ACCOUNT = ''
TOP_N = 3

In [ ]:
morphologies_name_mapping = {
    'old_classic_bipedal': 'bipedal',
    'climbing_profile_chimpanzee': 'chimpanzee',
    'fish': 'fish'
}

In [ ]:
for i,(m_id,label) in enumerate(labels.items()):
    push_top_seeds_to_hub(TOP_N, 
                      m_id, 
                      {
                        'student': 'SAC', 'environment': 'parkour', 'training_steps': 20000000, 'n_evaluation_tasks': 100
                      },
                      TOKEN, ACCOUNT
    )